See `utils.py` for all the helper functions used in this tutorial.

In [2]:
from utils import *

/home/van.bettauer/anaconda3/envs/open-mmlab-2/lib/python3.7/site-packages/mmcv/utils/registry.py:64: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


The first step is set the correct paths.  Make sure you have the data downloaded.  For the purposes of this tutorial, we will follow the file structure found at `csfg-algonquin.concordia.ca/~hallett/candescence/` (where you can also download the data).  We will assume you have the data downloaded into a folder called `candescence`.

The first path to set is the path to the config file used to train the model.  The config file contains all the parameters used for training the model, as well as additional architecture information.  For this tutorial, we will use the config file associated with our final model.  It can be found at `candescence/performance/exp27/training-of-model/exp27.py`

In [1]:
CONFIG_PATH = "candescence/performance/exp27/training-of-model/exp27.py"

Next, we need the path to the weights of the trained model.  It can be found at `candescence/performance/exp27/training-of-model/final.pth`

In [ ]:
MODEL_PATH = "candescence/performance/exp27/training-of-model/final.pth"

Finally, we need to provide the path to a directory containing the images you would like to run inference over. For this tutorial, we will use our validation images found at `candescence/varasana/train-validation/val/`

In [2]:
DATA_PATH = "candescence/varasana/train-validation/val/"